In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, concatenate, UpSampling2D, Multiply

# Define UNet architecture with selective attention mechanism
def unet_with_attention(input_shape, num_classes):
    inputs = Input(input_shape)

    import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, concatenate, UpSampling2D, Multiply

# Define UNet architecture with selective attention mechanism
def unet_with_attention(input_shape, num_classes):
    inputs = Input(input_shape)

    # Contracting Path (Encoder)
    c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(inputs)
    c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)

    # Add more contracting layers...

    # Bottom layer
    bn = Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(pN)
    bn = Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(bn)

    # Selective Attention Mechanism
    attention = Multiply()([bn, pN])  # Apply attention to bottom layer features

    # Expanding Path (Decoder)
    uN = UpSampling2D((2, 2))(attention)
    uN = concatenate([uN, cN], axis=-1)
    uN = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(uN)
    uN = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(uN)

    # Add more expanding layers...

    outputs = Conv2D(num_classes, (1, 1), activation='sigmoid')(uN)  # Output segmentation mask

    model = Model(inputs, outputs)
    return model


# Define Dice loss function
def dice_loss(y_true, y_pred):
    numerator = 2 * tf.reduce_sum(y_true * y_pred, axis=(1, 2, 3))
    denominator = tf.reduce_sum(y_true + y_pred, axis=(1, 2, 3))
    dice_loss = 1 - (numerator + 1) / (denominator + 1)
    return dice_loss

# Define hybrid loss function combining BCE and Dice loss
def hybrid_loss(y_true, y_pred):
    bce = tf.keras.losses.BinaryCrossentropy()(y_true, y_pred)
    dice = dice_loss(y_true, y_pred)
    hybrid_loss = bce + dice
    return hybrid_loss

# Load and preprocess your dataset

# Define input shape and number of classes
input_shape = (256, 256, 3)
num_classes = 1  # Binary segmentation

# Create and compile the model
model = unet_with_attention(input_shape, num_classes)
model.compile(optimizer='adam', loss=hybrid_loss, metrics=[dice_loss, 'accuracy'])

# Train your model

# Define evaluation function
def evaluate_model(model, test_data, test_labels):
    predictions = model.predict(test_data)
    
    # Calculate IoU, DSC, Precision, and Recall
    iou = tf.metrics.MeanIoU(num_classes=num_classes)(test_labels, predictions)
    dsc = dice_loss(test_labels, predictions)
    precision = tf.keras.metrics.Precision()(test_labels, predictions)
    recall = tf.keras.metrics.Recall()(test_labels, predictions)
    
    print(f"IoU: {iou}, DSC: {dsc}, Precision: {precision}, Recall: {recall}")

# Evaluate the model using test data and labels
evaluate_model(model, test_data, test_labels)


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, MaxPooling2D, AveragePooling2D, concatenate, UpSampling2D, Add, Multiply

# Define DeepLab-v3 architecture with selective attention mechanism
def deeplab_with_attention(input_shape, num_classes):
    inputs = Input(input_shape)

import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, MaxPooling2D, AveragePooling2D, concatenate, UpSampling2D, Add, Multiply

def _conv2d_bn_relu(input_tensor, filters, kernel_size, padding='same', dilation_rate=1):
    x = Conv2D(filters, kernel_size, padding=padding, dilation_rate=dilation_rate)(input_tensor)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    return x

def _aspp(input_tensor):
    d1 = _conv2d_bn_relu(input_tensor, 256, 1)
    d6 = _conv2d_bn_relu(input_tensor, 256, 3, dilation_rate=6)
    d12 = _conv2d_bn_relu(input_tensor, 256, 3, dilation_rate=12)
    d18 = _conv2d_bn_relu(input_tensor, 256, 3, dilation_rate=18)

    # Image-level features
    image_features = AveragePooling2D(pool_size=(input_tensor.shape[1], input_tensor.shape[2]))(input_tensor)
    image_features = _conv2d_bn_relu(image_features, 256, 1)

    # Concatenate all features
    concat_features = concatenate([d1, d6, d12, d18, image_features], axis=-1)
    aspp = _conv2d_bn_relu(concat_features, 256, 1)
    return aspp

def _residual_block(input_tensor, filters, kernel_size, strides=(1, 1)):
    shortcut = input_tensor
    x = _conv2d_bn_relu(input_tensor, filters, kernel_size, padding='same', dilation_rate=strides[0])
    x = _conv2d_bn_relu(x, filters, kernel_size, padding='same', dilation_rate=strides[0])
    shortcut = Conv2D(filters, (1, 1), padding='same', dilation_rate=strides[0])(shortcut)
    x = Add()([x, shortcut])
    return x

def deeplab_with_attention(input_shape, num_classes):
    inputs = Input(input_shape)

    # Entry block
    x = _conv2d_bn_relu(inputs, 32, 3, strides=(2, 2))
    x = _conv2d_bn_relu(x, 64, 3, strides=(1, 1))
    x = _residual_block(x, 64, 3)
    x = _conv2d_bn_relu(x, 128, 3, strides=(2, 2))

    # Middle block
    for _ in range(3):
        x = _residual_block(x, 128, 3)
    
    # Atrous Spatial Pyramid Pooling (ASPP)
    aspp = _aspp(x)

    # Decoder block
    x = UpSampling2D(size=(4, 4), interpolation='bilinear')(aspp)
    x = concatenate([x, inputs], axis=-1)
    x = _conv2d_bn_relu(x, 48, 3, strides=(1, 1))
    x = _conv2d_bn_relu(x, 48, 3, strides=(1, 1))
    x = Conv2D(num_classes, (1, 1), activation='sigmoid')(x)

    model = tf.keras.Model(inputs, x)
    return model


# Define Dice loss function
def dice_loss(y_true, y_pred):
    numerator = 2 * tf.reduce_sum(y_true * y_pred, axis=(1, 2, 3))
    denominator = tf.reduce_sum(y_true + y_pred, axis=(1, 2, 3))
    dice_loss = 1 - (numerator + 1) / (denominator + 1)
    return dice_loss

# Define hybrid loss function combining BCE and Dice loss
def hybrid_loss(y_true, y_pred):
    bce = tf.keras.losses.BinaryCrossentropy()(y_true, y_pred)
    dice = dice_loss(y_true, y_pred)
    hybrid_loss = bce + dice
    return hybrid_loss

# Load and preprocess your dataset

# Define input shape and number of classes
input_shape = (256, 256, 3)
num_classes = 1  # Binary segmentation

# Create and compile the model
model = deeplab_with_attention(input_shape, num_classes)
model.compile(optimizer='adam', loss=hybrid_loss, metrics=[dice_loss, 'accuracy'])

# Train your model

# Define evaluation function
def evaluate_model(model, test_data, test_labels):
    predictions = model.predict(test_data)
    
    # Calculate IoU, DSC, Precision, and Recall
    iou = tf.metrics.MeanIoU(num_classes=num_classes)(test_labels, predictions)
    dsc = dice_loss(test_labels, predictions)
    precision = tf.keras.metrics.Precision()(test_labels, predictions)
    recall = tf.keras.metrics.Recall()(test_labels, predictions)
    
    print(f"IoU: {iou}, DSC: {dsc}, Precision: {precision}, Recall: {recall}")

# Evaluate the model using test data and labels
evaluate_model(model, test_data, test_labels)


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Multiply

# Define FCN architecture with selective attention mechanism
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Multiply

def _conv2d_bn_relu(input_tensor, filters, kernel_size, padding='same'):
    x = Conv2D(filters, kernel_size, padding=padding)(input_tensor)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    return x

def fcn_with_attention(input_shape, num_classes):
    inputs = Input(input_shape)

    # Encoder
    c1 = _conv2d_bn_relu(inputs, 64, 3)
    c1 = _conv2d_bn_relu(c1, 64, 3)
    p1 = MaxPooling2D((2, 2))(c1)

    # Add more encoder layers...

    # Decoder
    uN = UpSampling2D((2, 2))(pN)
    uN = concatenate([uN, cN], axis=-1)
    uN = _conv2d_bn_relu(uN, 128, 3)
    uN = _conv2d_bn_relu(uN, 128, 3)

    # Selective Attention Mechanism
    attention = Multiply()([uN, pN])  # Apply attention to decoder features

    # Final convolution layer
    outputs = Conv2D(num_classes, (1, 1), activation='sigmoid')(attention)

    model = Model(inputs, outputs)
    return model


# Define Dice loss function
def dice_loss(y_true, y_pred):
    numerator = 2 * tf.reduce_sum(y_true * y_pred, axis=(1, 2, 3))
    denominator = tf.reduce_sum(y_true + y_pred, axis=(1, 2, 3))
    dice_loss = 1 - (numerator + 1) / (denominator + 1)
    return dice_loss

# Define hybrid loss function combining BCE and Dice loss
def hybrid_loss(y_true, y_pred):
    bce = tf.keras.losses.BinaryCrossentropy()(y_true, y_pred)
    dice = dice_loss(y_true, y_pred)
    hybrid_loss = bce + dice
    return hybrid_loss

# Load and preprocess your dataset

# Define input shape and number of classes
input_shape = (256, 256, 3)
num_classes = 1  # Binary segmentation

# Create and compile the model
model = fcn_with_attention(input_shape, num_classes)
model.compile(optimizer='adam', loss=hybrid_loss, metrics=[dice_loss, 'accuracy'])

# Train your model

# Define evaluation function
def evaluate_model(model, test_data, test_labels):
    predictions = model.predict(test_data)
    
    # Calculate IoU, DSC, Precision, and Recall
    iou = tf.metrics.MeanIoU(num_classes=num_classes)(test_labels, predictions)
    dsc = dice_loss(test_labels, predictions)
    precision = tf.keras.metrics.Precision()(test_labels, predictions)
    recall = tf.keras.metrics.Recall()(test_labels, predictions)
    
    print(f"IoU: {iou}, DSC: {dsc}, Precision: {precision}, Recall: {recall}")

# Evaluate the model using test data and labels
evaluate_model(model, test_data, test_labels)
